In [2]:
import pandas as pd
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

In [3]:
# read the CSV file
df = pd.read_csv("D:\\SensViz\\Machine Learning Task 2\\train.csv")
# separate the label column
labels = df['label']
# separate the pixel data columns and convert them into a numpy array
pixels = df.iloc[:, 1:].to_numpy(dtype='uint8')
# create the output directory if it does not exist
output_dir = "D:\SensViz\Machine Learning Task 2\images"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
# loop over the pixel data and convert each row into an image
for i in range(pixels.shape[0]):
    pixel_data = pixels[i].reshape(28, 28)
    img = cv2.cvtColor(pixel_data, cv2.COLOR_GRAY2BGR)  # convert the grayscale image to BGR
    filename = os.path.join(output_dir, f"{i}.jpg")
    cv2.imwrite(filename, img)

In [4]:
# reshape the pixel data and convert it into float values between 0 and 1
pixels = pixels.reshape(-1, 28, 28, 1) / 255.0

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(pixels, labels, test_size=0.1, random_state=42)

# build the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='elu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='elu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

Epoch 1/5
1182/1182 [==============================] - 40s 33ms/step - loss: 0.2004 - accuracy: 0.9376 - val_loss: 0.0831 - val_accuracy: 0.9736
Epoch 2/5
1182/1182 [==============================] - 36s 30ms/step - loss: 0.0825 - accuracy: 0.9745 - val_loss: 0.0654 - val_accuracy: 0.9800
Epoch 3/5
1182/1182 [==============================] - 41s 34ms/step - loss: 0.0652 - accuracy: 0.9793 - val_loss: 0.0519 - val_accuracy: 0.9845
Epoch 4/5
1182/1182 [==============================] - 50s 43ms/step - loss: 0.0575 - accuracy: 0.9820 - val_loss: 0.0506 - val_accuracy: 0.9855
Epoch 5/5
132/132 [==============================] - 1s 10ms/step - loss: 0.0498 - accuracy: 0.9852
Test accuracy: 0.9852380752563477


In [12]:
test_data = pd.read_csv("D:\\SensViz\\Machine Learning Task 2\\test.csv")
test_pixels = test_data.to_numpy(dtype='uint8')
test_images = test_pixels.reshape(test_pixels.shape[0], 28, 28, 1) / 255.0

predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)
sample_submission = pd.read_csv("D:\\SensViz\\Machine Learning Task 2\\sample_submission.csv")
sample_submission.head()
sample_submission['Label']= predicted_labels


875/875 [==============================] - 11s 12ms/step


In [29]:
sample_submission = pd.read_csv('sample_submission.csv')
modified_sample_submission = sample_submission[['ImageId', 'Label']]
modified_sample_submission.to_csv('modified_sample_submission.csv', index=False)

df = pd.read_csv('modified_sample_submission.csv')
df.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
